In [23]:
import tensorflow as tf

In [24]:
import numpy as np
import time

In [25]:
with open('letters_source.txt', 'r', encoding = 'utf-8') as f:
    source_data = f.read()
    
with open('letters_target.txt', 'r', encoding = 'utf-8') as f:
    target_data = f.read()

In [26]:
source_data.split('\n')[:10]

['bsaqq',
 'npy',
 'lbwuj',
 'bqv',
 'kial',
 'tddam',
 'edxpjpg',
 'nspv',
 'huloz',
 'kmclq']

In [27]:
target_data.split('\n')[:10]

['abqqs',
 'npy',
 'bjluw',
 'bqv',
 'aikl',
 'addmt',
 'degjppx',
 'npsv',
 'hlouz',
 'cklmq']

In [28]:
def extrtact_character_vocab(data):
    #construct mapping table
    special_words = ['<PAD>','<UNK>','<GO>','<EOS>']
    set_words = list(set([char for line in data.split('\n') for char in line]))
    
    #add the special words in vocab
    int_to_vocab = {idx : word for idx, word in enumerate(special_words + set_words)}
    vocab_to_int = {word : idx for idx, word in int_to_vocab.items()}
    
    return int_to_vocab, vocab_to_int

In [29]:
#mapping table
source_int_to_letter, source_letter_to_int = extrtact_character_vocab(source_data)
target_int_to_letter, target_letter_to_int = extrtact_character_vocab(target_data)

#convert character
source_int = [[source_letter_to_int.get(letter, source_letter_to_int['<UNK>']) 
              for letter in line] for line in source_data.split('\n')]#DISPLAY EACH WORD IN VOCAB IN FORM OF CHAR, 
#THEN TRANSLATE TO INT, UNK is default return value for get in dict
target_int = [[target_letter_to_int.get(letter, target_letter_to_int['<UNK>'])
              for letter in line] + [target_letter_to_int['<EOS>']] for line in target_data.split('\n')]
#<EOS> at the end of every word

In [30]:
source_int[:10]

[[21, 9, 13, 24, 24],
 [6, 29, 19],
 [11, 21, 4, 12, 7],
 [21, 24, 17],
 [27, 26, 13, 11],
 [20, 22, 22, 13, 15],
 [28, 22, 5, 29, 7, 29, 16],
 [6, 9, 29, 17],
 [8, 12, 11, 25, 14],
 [27, 15, 23, 11, 24]]

In [31]:
target_int[:10]

[[13, 21, 24, 24, 9, 3],
 [6, 29, 19, 3],
 [21, 7, 11, 12, 4, 3],
 [21, 24, 17, 3],
 [13, 26, 27, 11, 3],
 [13, 22, 22, 15, 20, 3],
 [22, 28, 16, 7, 29, 29, 5, 3],
 [6, 29, 9, 17, 3],
 [8, 11, 25, 12, 14, 3],
 [23, 27, 11, 15, 24, 3]]

In [32]:
def get_inputs():
    '''
    input tensor of model
    '''
    inputs = tf.placeholder(tf.int32, [None, None], name='inputs')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    

    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return inputs, targets, learning_rate, target_sequence_length, max_target_sequence_length, source_sequence_length

In [33]:
def get_encoder_layer(input_data, rnn_size, num_layers, source_sequence_length, 
                      source_vocab_size, encoding_embedding_size):
    #Encoder embedding
    encoder_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)
    
    #RNN Cell
    def get_lstm_cell(rnn_size):
        lstm_cell = tf.contrib.rnn.LSTMCell(rnn_size, initializer = tf.random_uniform_initializer(-0.1, 0.1, seed = 2))
        return lstm_cell
    cell = tf.contrib.rnn.MultiRNNCell([get_lstm_cell(rnn_size) for _ in range(num_layers)])
    
    encoder_output, encoder_state = tf.nn.dynamic_rnn(cell, encoder_embed_input,
                                                     sequence_length = source_sequence_length, dtype = tf.float32)
    
    return encoder_output, encoder_state    

In [34]:
def process_decoder_input(data, vocab_to_int, batch_size):
    
    ending = tf.strided_slice(data, [0, 0], [batch_size, -1], [1, 1])
    decoder_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)
    
    return decoder_input

In [35]:

def decoding_layer(target_letter_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, encoder_state, decoder_input):

    # 1. Embedding
    target_vocab_size = len(target_letter_to_int)
    decoder_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    decoder_embed_input = tf.nn.embedding_lookup(decoder_embeddings, decoder_input)

    # 2. Construct RNN Cells in Decoder
    def get_decoder_cell(rnn_size):
        decoder_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return decoder_cell
    cell = tf.contrib.rnn.MultiRNNCell([get_decoder_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Output FC layer
    output_layer = tf.layers.Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))


    # 4. Training decoder
    with tf.variable_scope("decode"):
        # get object of help
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=decoder_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        # construct decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                           training_helper,
                                                           encoder_state,
                                                           output_layer) 
        training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder, 
                                                                  impute_finished=True, 
                                                                  maximum_iterations=max_target_sequence_length)
    # 5. Predicting decoder
    # share param with training
    with tf.variable_scope("decode", reuse=True):
        start_tokens = tf.tile(tf.constant([target_letter_to_int['<GO>']], dtype=tf.int32), [batch_size], 
                               name='start_tokens')
        predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(decoder_embeddings,
                                                                start_tokens,
                                                                target_letter_to_int['<EOS>'])
        predicting_decoder = tf.contrib.seq2seq.BasicDecoder(cell,
                                                        predicting_helper,
                                                        encoder_state,
                                                        output_layer)
        predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(predicting_decoder, impute_finished=True,
                                                                        maximum_iterations=max_target_sequence_length)
    
    return training_decoder_output, predicting_decoder_output

In [36]:
def seq2seq_model(input_data, targets, lr, target_sequence_length, 
                  max_target_sequence_length, source_sequence_length,
                  source_vocab_size, target_vocab_size,
                  encoder_embedding_size, decoder_embedding_size, 
                  rnn_size, num_layers):
    
    # aquire encoder state output
    _, encoder_state = get_encoder_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # decoder's input after preprocessing
    decoder_input = process_decoder_input(targets, target_letter_to_int, batch_size)
    
    # state and input to decoder
    training_decoder_output, predicting_decoder_output = decoding_layer(target_letter_to_int, 
                                                                       decoding_embedding_size, 
                                                                       num_layers, 
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       encoder_state, 
                                                                       decoder_input) 
    
    return training_decoder_output, predicting_decoder_output

In [37]:
#hyper
# Number of Epochs
epochs = 60
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.001

In [38]:
# construct graph
train_graph = tf.Graph()

with train_graph.as_default():
    
    # get model's input   
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_inputs()
    
    training_decoder_output, predicting_decoder_output = seq2seq_model(input_data, 
                                                                      targets, 
                                                                      lr, 
                                                                      target_sequence_length, 
                                                                      max_target_sequence_length, 
                                                                      source_sequence_length,
                                                                      len(source_letter_to_int),
                                                                      len(target_letter_to_int),
                                                                      encoding_embedding_size, 
                                                                      decoding_embedding_size, 
                                                                      rnn_size, 
                                                                      num_layers)    
    
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits')
    predicting_logits = tf.identity(predicting_decoder_output.sample_id, name='predictions')
    
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

In [39]:
def pad_sentence_batch(sentence_batch, pad_int):
    #padding
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]

In [40]:
def get_batches(targets, sources, batch_size, source_pad_int, target_pad_int):
    #in order to get batch
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        #padding
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))
        
        # record each len
        targets_lengths = []
        for target in targets_batch:
            targets_lengths.append(len(target))
        
        source_lengths = []
        for source in sources_batch:
            source_lengths.append(len(source))
        
        yield pad_targets_batch, pad_sources_batch, targets_lengths, source_lengths

In [41]:
#train and vali
train_source = source_int[batch_size:]
train_target = target_int[batch_size:]
# one batch to validate
valid_source = source_int[:batch_size]
valid_target = target_int[:batch_size]
(valid_targets_batch, valid_sources_batch, valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target, valid_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>']))

display_step = 50 # each 50 times print loss

checkpoint = "trained_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in enumerate(
                get_batches(train_target, train_source, batch_size,
                           source_letter_to_int['<PAD>'],
                           target_letter_to_int['<PAD>'])):
            
            _, loss = sess.run(
                [train_op, cost],
                {input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths})

            if batch_i % display_step == 0:
                
                # calculate validation loss
                validation_loss = sess.run(
                [cost],
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 lr: learning_rate,
                 target_sequence_length: valid_targets_lengths,
                 source_sequence_length: valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Training Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              loss, 
                              validation_loss[0]))

    
    
    # save model
    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')

Epoch   1/60 Batch    0/77 - Training Loss:  3.401  - Validation loss:  3.396
Epoch   1/60 Batch   50/77 - Training Loss:  2.910  - Validation loss:  2.907
Epoch   2/60 Batch    0/77 - Training Loss:  2.556  - Validation loss:  2.560
Epoch   2/60 Batch   50/77 - Training Loss:  2.168  - Validation loss:  2.187
Epoch   3/60 Batch    0/77 - Training Loss:  2.042  - Validation loss:  2.032
Epoch   3/60 Batch   50/77 - Training Loss:  1.830  - Validation loss:  1.827
Epoch   4/60 Batch    0/77 - Training Loss:  1.749  - Validation loss:  1.731
Epoch   4/60 Batch   50/77 - Training Loss:  1.590  - Validation loss:  1.564
Epoch   5/60 Batch    0/77 - Training Loss:  1.493  - Validation loss:  1.487
Epoch   5/60 Batch   50/77 - Training Loss:  1.399  - Validation loss:  1.358
Epoch   6/60 Batch    0/77 - Training Loss:  1.315  - Validation loss:  1.297
Epoch   6/60 Batch   50/77 - Training Loss:  1.239  - Validation loss:  1.183
Epoch   7/60 Batch    0/77 - Training Loss:  1.159  - Validation

Epoch  54/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.011
Epoch  54/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.011
Epoch  55/60 Batch    0/77 - Training Loss:  0.006  - Validation loss:  0.010
Epoch  55/60 Batch   50/77 - Training Loss:  0.006  - Validation loss:  0.010
Epoch  56/60 Batch    0/77 - Training Loss:  0.005  - Validation loss:  0.010
Epoch  56/60 Batch   50/77 - Training Loss:  0.005  - Validation loss:  0.009
Epoch  57/60 Batch    0/77 - Training Loss:  0.005  - Validation loss:  0.009
Epoch  57/60 Batch   50/77 - Training Loss:  0.005  - Validation loss:  0.009
Epoch  58/60 Batch    0/77 - Training Loss:  0.005  - Validation loss:  0.009
Epoch  58/60 Batch   50/77 - Training Loss:  0.005  - Validation loss:  0.008
Epoch  59/60 Batch    0/77 - Training Loss:  0.004  - Validation loss:  0.008
Epoch  59/60 Batch   50/77 - Training Loss:  0.004  - Validation loss:  0.008
Epoch  60/60 Batch    0/77 - Training Loss:  0.004  - Validation

In [42]:
def source_to_seq(text):
    '''
    convert source data
    '''
    sequence_length = 7
    return [source_letter_to_int.get(word, 
                                     source_letter_to_int['<UNK>']) for word in text] + [source_letter_to_int['<PAD>']]*(sequence_length-len(text))

In [43]:
input_word = 'zhengjiapengniubi'
text = source_to_seq(input_word)

checkpoint = "./trained_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # load model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('inputs:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(input_word)]*batch_size, 
                                      source_sequence_length: [len(input_word)]*batch_size})[0] 


pad = source_letter_to_int["<PAD>"] 

print('Original INPUT:', input_word)

print('\nSource')
print('  Word Num.:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

print('\nTarget')
print('  Word Num.:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))

Original INPUT: zhengjiapengniubi

Source
  Word Num.:    [14, 8, 28, 6, 16, 7, 26, 13, 29, 28, 6, 16, 6, 26, 12, 21, 26]
  Input Words: z h e n g j i a p e n g n i u b i

Target
  Word Num.:       [13, 21, 28, 28, 16, 8, 26, 26, 26, 7, 6, 6, 6, 29, 17, 5, 3]
  Response Words: a b e e g h i i i j n n n p v x <EOS>
